# Notebook 3: Interactive Dashboard for Energy Import Dependency

# Overview
This notebook presents a simple interactive dashboard to support intuitive exploration of energy import dependency trends.

# Key Features
- Select a country and year range
- Compare selected countries with Ireland
- Visualise trends and rolling averages
- Optionally save selected plots for reporting

# Purpose
The dashboard is designed to complement the analytical notebooks by improving interpretability and accessibility of results for both technical and non-technical audiences.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline



BASE = Path.cwd()
ROOT = BASE.parent if BASE.name == "notebooks" else BASE
DATA = ROOT / "data" / "processed" / "import_dependency_clean.csv"
FIGS = ROOT / "figures"
FIGS.mkdir(exist_ok=True)

df = pd.read_csv(DATA)[["geo","year","value"]].dropna()
df["year"] = df["year"].astype(int)
df = df.sort_values(["geo","year"])

countries = sorted(df["geo"].unique())
min_y, max_y = df["year"].min(), df["year"].max()


country = widgets.Dropdown(
    options=[c for c in countries if c != "IE"],
    value="FR" if "FR" in countries else countries[0],
    description="Country:",
    style={"description_width": "initial"}
)

years = widgets.IntRangeSlider(
    value=[min_y, max_y],
    min=min_y, max=max_y,
    step=1,
    description="Years:",
    continuous_update=False,
    layout=widgets.Layout(width="500px"),
    style={"description_width": "initial"}
)

save_btn = widgets.Button(description="Save image", button_style="success")
msg = widgets.Label("")
out = widgets.Output()

display(widgets.VBox([
    widgets.HBox([country, years, save_btn, msg]),
    out
]))

def draw(*_):
    out.clear_output(wait=True)
    plt.close("all")

    c = country.value
    y0, y1 = years.value

    d = df[(df["geo"] == c) & (df["year"].between(y0, y1))].copy()
    ie = df[(df["geo"] == "IE") & (df["year"].between(y0, y1))].copy()

    if d.empty or ie.empty:
        with out:
            print("No data available.")
        return

    
    d["roll5"] = d["value"].rolling(5, min_periods=1).mean()

    fig, axes = plt.subplots(2, 1, figsize=(8, 5), sharex=True)

    
    axes[0].plot(d["year"], d["value"], label="Actual")
    axes[0].plot(d["year"], d["roll5"], linestyle="--", label="5-year trend")
    axes[0].set_title(f"{c} — Import Dependency (%)")
    axes[0].set_ylabel("%")
    axes[0].legend(frameon=False)

    
    axes[1].plot(d["year"], d["value"], label=c)
    axes[1].plot(ie["year"], ie["value"], linestyle="--", label="Ireland (IE)")
    axes[1].set_title(f"{c} vs Ireland")
    axes[1].set_xlabel("Year")
    axes[1].set_ylabel("%")
    axes[1].legend(frameon=False)

    for ax in axes:
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.grid(axis="y", linestyle=":", alpha=0.3)

    with out:
        plt.show()
        plt.close(fig)



def save_current(_=None):
    c = country.value
    y0, y1 = years.value

    d = df[(df["geo"] == c) & (df["year"].between(y0, y1))].copy()
    ie = df[(df["geo"] == "IE") & (df["year"].between(y0, y1))].copy()

    if d.empty or ie.empty:
        msg.value = "Nothing to save."
        return

    d["roll5"] = d["value"].rolling(5, min_periods=1).mean()

    fig, axes = plt.subplots(2, 1, figsize=(8, 5), sharex=True)

    axes[0].plot(d["year"], d["value"], label="Actual")
    axes[0].plot(d["year"], d["roll5"], linestyle="--", label="5-year trend")
    axes[0].set_title(f"{c} — Import Dependency (%)")
    axes[0].set_ylabel("%")
    axes[0].legend(frameon=False)

    axes[1].plot(d["year"], d["value"], label=c)
    axes[1].plot(ie["year"], ie["value"], linestyle="--", label="Ireland (IE)")
    axes[1].set_title(f"{c} vs Ireland")
    axes[1].set_xlabel("Year")
    axes[1].set_ylabel("%")
    axes[1].legend(frameon=False)

    for ax in axes:
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.grid(axis="y", linestyle=":", alpha=0.3)

    out_path = FIGS / f"dashboard_{c}_{y0}_{y1}.png"
    fig.tight_layout()
    fig.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close(fig)

    msg.value = f"Saved: {out_path.name}"




country.observe(draw, names="value")
years.observe(draw, names="value")
draw()

save_btn._click_handlers.callbacks = []
save_btn.on_click(save_current)


# Summary

This notebook provides an interactive visual interface for exploring energy import dependency data.  
It allows users to dynamically compare countries and time periods, reinforcing insights obtained from exploratory, statistical, and predictive analyses.

The dashboard supports transparent communication of results and serves as a final presentation layer for the project.
